In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('names.txt').read().splitlines()

In [3]:
chrs = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chrs)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(itos)
print(f'Vocab size: {vocab_size}')

Vocab size: 27


In [4]:
block_size = 3

def build_dataset(ds):
    X, Y = [], []

    for word in ds:
        context = [0] * block_size
        for ch in word + '.':
            chi = stoi[ch]
            X.append(context)
            Y.append(chi)
            context = context[1:] + [chi]
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

# splitting
import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [13]:
# compare torch grad vs manual grad
def cmp(s, dt, t): # layer - derivative - tensor
  ex = torch.all(dt == t.grad).item() # if all equals
  app = torch.allclose(dt, t.grad) # if all close
  maxdiff = (dt - t.grad).abs().max().item() # maximum difference
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [14]:
n_emd = 10
n_hidden = 64

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_emd), generator=g)
# L1
W1 = torch.randn((n_emd * block_size, n_hidden), generator=g) * (5/3)/((n_emd * block_size)**0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1
# L2
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1
# batchnorm
bngain = torch.randn((1, n_hidden), generator=g) * 0.1 + 1.0
bnbias = torch.randn((1, n_hidden), generator=g) * 0.1

parameters = [C, W1, b2, W2, b2, bngain, bnbias]
for p in parameters:
    p.requires_grad = True

In [15]:
# minibatch
batch_size = 32
n = batch_size
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

In [16]:
# forward pass
emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer preactviation
# BatchNorm layer
bnmeani = 1/n * hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff ** 2
bnvar = 1 / (n-1) * (bndiff2).sum(0, keepdim=True) # Bessel's correction: biased vs unbiased estimate
bnvar_inv = (bnvar + 1e-5) **-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-Linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2
# cross entropy loss
logits_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logits_maxes # bc of exp() logits could become large values, can cause instability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdim=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

In [17]:
# pytorch backward pass
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, norm_logits, logits_maxes, logits, h, hpreact, bnraw, bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani, embcat, emb]:
    t.retain_grad()
loss.backward()
loss

tensor(3.5571, grad_fn=<NegBackward0>)

In [21]:
# manual backward pass

# loss = -1/na + -1/nb + -1/nc
# dloss/da = -1/n
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0/n
cmp('logprobs', dlogprobs, logprobs)

# d/dx log(x) = 1 / x
dprobs = (1.0 / probs) * dlogprobs
cmp('probs', dprobs, probs)

dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)

dcounts = counts_sum_inv * dprobs

# d/dx x**-1 = - x ** -2
dcounts_sum = (-counts_sum ** -2) * dcounts_sum_inv
cmp('counts_sum', dcounts_sum, counts_sum)

dcounts += torch.ones_like(counts) * dcounts_sum
cmp('counts', dcounts, counts)

# d/dx exp(x) = exp(x)
dnorm_logits = (norm_logits.exp()) * dcounts
cmp('norm_logits', dnorm_logits, norm_logits)

dlogits = dnorm_logits.clone()
dlogits_maxes = -dnorm_logits.sum(1, keepdim=True)
cmp('logits_maxes', dlogits_maxes, logits_maxes)

dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogits_maxes
cmp('logits', dlogits, logits)

# dlogits (32, 27), h(32, 64), W2(64, 27), b2(27)
dh = dlogits @ W2.T # shape of dh is the same as the shape of h
dW2 = h.T @ dlogits
db2 = dlogits.sum(0) # summing on the 0th dimension, so the shape can be (27)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)

# dx/d tanh(x) = 1 - x**2
dhpreact = (1.0 - h**2) * dh
cmp('hpreact', dhpreact, hpreact)

# a * b = c -> dx/d = b
# bngain (1, 64), output would be (32, 64)
dbngain = (bnraw * dhpreact).sum(0, keepdim=True) # keeping dim because output should be (1, 64), not (64)
dbnraw = bngain * dhpreact
dbnbias = dhpreact.sum(0, keepdim=True)
cmp('bngain', dbngain, bngain)
cmp('bnraw', dbnraw, bnraw)
cmp('bnbias', dbnbias, bnbias)

dbndiff = bnvar_inv * dbnraw
# would (32,64) so summing at 0th dimension
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)

# dx/d x**n = n * x**n-1 and chain rule
dbnvar = (-0.5*(bnvar + 1e-5) ** -1.5) * dbnvar_inv
cmp('bnvar', dbnvar, bnvar)

# (1.0/(n-1)) is the derivative of bnvar line
dbndiff2 = torch.ones_like(bndiff2) * (1.0/(n-1)) * dbnvar
cmp('bndiff2', dbndiff2, bndiff2)

dbndiff += (2 * bndiff) * dbndiff2
cmp('bndiff', dbndiff, bndiff)

# bndiff and hprebn (32, 64) but bnmeani (1, 64), so .sum()
dhprebn = dbndiff.clone()
dbnmeani = (-dbndiff).sum(0) # broadcasting in forward pass => summing in backward pass
cmp('bnmeani', dbnmeani, bnmeani)

# scaling and broadcasting
dhprebn += torch.ones_like(hprebn) * (1.0/n * dbnmeani)
cmp('hprebn', dhprebn, hprebn)

# hprebn(32, 64), embcat(32, 30), W1(30, 64), b1(64)
dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0) # keepdim=False to eliminate first dimension
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
# cmp('b1', db1, b1)

# undoing view, viewing it as original
demb = dembcat.view(emb.shape)
cmp('emb', demb, emb)

# emb (32,3,10), C (27,10), Xb (32,3)
# undoing indexing, each occurence means addition
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[k,j]
        dC[ix] += demb[k,j] # += because of multiple occurences
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logits_maxes    | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: True  | approximate: True  | maxdiff: 0.0
bngain          | exact: True  | approximate: True  | maxdiff: 0.0
bnraw           | exact: True  | approximate: True  | maxdiff: 0.0
bnbias          | exact: True  | approximate: True  | maxdiff:

In [22]:
# cross entropy simplified
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff: ', (loss_fast-loss).item())

3.5571467876434326 diff:  -2.384185791015625e-07


In [23]:
# backward pass with simplified loss function

#dloss/dli of softmax
dlogits = F.softmax(logits, 1)
dlogits[range(n), Yb] -= 1
dlogits /= n # because of mean it needs to be scaled down
cmp('logits', dlogits, logits)

logits          | exact: False | approximate: True  | maxdiff: 5.122274160385132e-09


In [24]:
# batchnorm simplified
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

max diff: tensor(4.7684e-07, grad_fn=<MaxBackward1>)


In [25]:
# backward pass with simplified batchnorm

dhprebn = bngain * bnvar_inv/n * (n * dhpreact - dhpreact.sum(0) - n / (n-1) * bnraw * (dhpreact * bnraw).sum(0))
cmp('hprebn', dhprebn, hprebn)

hprebn          | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10


In [26]:
# building a MLP with manual backward pass
n_emdb = 10
n_hidden = 200

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_emd), generator=g)
# L1
W1 = torch.randn((n_emd * block_size, n_hidden), generator=g) * (5/3)/((n_emd * block_size)**0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1
# L2
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1
# BatchNorm
bngain = torch.randn((1, n_hidden)) * 0.1 + 1.0
bnbias = torch.randn((1, n_hidden)) * 0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

max_steps = 200000
batch_size = 32
n = batch_size
lossi = []

with torch.no_grad():
    for i in range(max_steps):
        ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
        Xb, Yb = Xtr[ix], Ytr[ix]

        # forward
        emb = C[Xb]
        embcat = emb.view(emb.shape[0], -1)
        hprebn = embcat @ W1 + b1
        bnmean = hprebn.mean(0, keepdim=True)
        bnvar = hprebn.var(0, keepdim=True, unbiased=True)
        bnvar_inv = (bnvar + 1e-5)**-0.5
        bnraw = (hprebn - bnmean) * bnvar_inv
        hpreact = bngain * bnraw + bnbias
        h = torch.tanh(hpreact)
        logits = h @ W2 + b2
        loss = F.cross_entropy(logits, Yb)

        # backward
        for p in parameters:
            p.grad = None

        # manual backprop
        dlogits = F.softmax(logits, 1)
        dlogits[range(n), Yb] -= 1
        dlogits /= n
        # 2nd layer
        dh = dlogits @ W2.T
        dW2 = h.T @ dlogits
        db2 = dlogits.sum(0)
        # tanh
        dhpreact = (1.0 - h**2) * dh
        # batchnorm backprop
        dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
        dbnbias = dhpreact.sum(0, keepdim=True)
        dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))
        # 1st layer
        dembcat = dhprebn @ W1.T
        dW1 = embcat.T @ dhprebn
        db1 = dhprebn.sum(0)
        # embedding
        demb = dembcat.view(emb.shape)
        dC = torch.zeros_like(C)
        for k in range(Xb.shape[0]):
            for j in range(Xb.shape[1]):
                ix = Xb[k,j]
                dC[ix] += demb[k,j]
        grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]

    
        # update
        lr = 0.1 if i < 100000 else 0.01
        for p, grad in zip(parameters, grads):
            p.data += -lr * grad
        
        # stats
        if i % 10000 == 0:
            print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
        lossi.append(loss.log10().item())
        
        # if i >= 100:
        #     break

12297
      0/ 200000: 3.8202


In [ ]:
with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnvar = hpreact.var(0, keepdim=True, unbiased=True)

In [ ]:
@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 + b1
  hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

In [ ]:
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # ------------
      # forward pass:
      # Embedding
      emb = C[torch.tensor([context])] # (1,block_size,d)      
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # ------------
      # Sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))
